# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [16]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [17]:
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [18]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [19]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [20]:
from cassandra.cluster import Cluster

cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [21]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [22]:
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Query 1 : Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [23]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

session.execute(""" CREATE TABLE IF NOT EXISTS songs_listened_by_session (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))""")
                    

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_listened_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [25]:
rows = session.execute(""" SELECT artist, song, length FROM songs_listened_by_session WHERE sessionID=338 AND itemInSession=4 """)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2 : Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [26]:
session.execute("""CREATE TABLE IF NOT EXISTS songs_listened_by_user (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, 
PRIMARY KEY (userId, sessionId, itemInSession)) WITH CLUSTERING ORDER BY (sessionId ASC, itemInSession ASC)""")
                    

In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_listened_by_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]), line[0], line[9], line[1], line[4]))

In [28]:
rows = session.execute(""" SELECT artist, song, firstName, lastName FROM songs_listened_by_user
WHERE userId=10 AND sessionid=182""")

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3 : Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [29]:
session.execute("""CREATE TABLE IF NOT EXISTS users_listening_given_song (firstName text, lastName text, song text, 
PRIMARY KEY (firstName, lastName, song))""")

session.execute("""CREATE INDEX ON users_listening_given_song (song)""")

In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_listening_given_song (firstName, lastName, song)"
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9]))

In [31]:
rows = session.execute(""" SELECT firstName, lastName FROM users_listening_given_song 
WHERE song='All Hands Against His Own' 
""")

for row in rows:
    print (row.firstname, row.lastname)

Tegan Levine
Jacqueline Lynch
Sara Johnson


### Drop the tables before closing out the sessions

In [32]:
session.execute("""DROP TABLE IF EXISTS songs_listened_by_session""")
session.execute("""DROP TABLE IF EXISTS songs_listened_by_user""")
session.execute("""DROP TABLE IF EXISTS users_listening_given_song""")

### Close the session and cluster connection¶

In [33]:
session.shutdown()
cluster.shutdown()